# MLSD : Assignment 1

### By: Alexandra de Carvalho, nmec 93346

In [1]:
from pyspark import SparkContext
import re
import random

In [3]:
sc = SparkContext(appName="Assignment1")

22/04/23 23:37:41 WARN Utils: Your hostname, jarvis resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlp4s0)
22/04/23 23:37:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/23 23:37:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Exercise 2

In [40]:
k = 3

data = sc.textFile('dataset2/plot_summaries.txt')                                                                                # reading the dataset
data = data.map(lambda plot: tuple(plot.split("\t")))                                                                            # (movie id, plot)
data = data.map(lambda movie_plot: (movie_plot[0], re.sub(r'[^\w\s]', '', movie_plot[1].lower())))                               # removing punctuation and case-sensitiveness from plot
data = data.map(lambda movie_plot: (movie_plot[0], movie_plot[1].split()))                                                       # (movie id, [list of words in plot]) 
data = data.map(lambda movie_plot: (movie_plot[0], {" ".join(movie_plot[1][i:i+k]) for i in range(len(movie_plot[1])-k + 1)}))   # (movie id, {k-shingles})

In [41]:
data = sc.parallelize(data.take(100))

In [42]:
vocab = data.flatMap(lambda movie_shingles: movie_shingles[1]).distinct().collect()

In [43]:
data = data.map(lambda movie_shingles: (movie_shingles[0], sorted([vocab.index(shingle) for shingle in movie_shingles[1]])))

In [44]:
for r in range(150):
    for b in range (150):
        if (1-(0.8**r))**b <= (0.5/100) and 1-(1-(0.4)**r)**b < 5/100:
            print(r,b)
            break

7 23
8 29
9 37
10 47
11 59
12 75
13 94
14 118
15 148


In [45]:
b = 23
r = 7
n = b*r
    
hash_parameters = [(random.randint(1,n**2), random.randint(0,n**2)) for _ in range(n)]

In [46]:
def get_signature(movie_shingleidx_signature:tuple):
    signature = movie_shingleidx_signature[2]
    for shingle in movie_shingleidx_signature[1]:
        signature = [min(((hash_parameters[h_idx][0]*shingle + hash_parameters[h_idx][1])% 2147483647) % (2**16), signature[h_idx]) for h_idx in range(len(hash_parameters))]

    return (movie_shingleidx_signature[0], signature)

In [47]:
signatures = data.map(lambda movie_1hot: (movie_1hot[0], movie_1hot[1][1:], [((hash_parameters[h_idx][0]*movie_1hot[1][0] + hash_parameters[h_idx][1]) % 2147483647) % (2**16) for h_idx in range(len(hash_parameters))]))
signatures = signatures.map(get_signature)

In [48]:
documents_lsh = signatures.map(lambda movie_signature: (movie_signature[0], [movie_signature[1][i:i+int(r)] for i in range(0, len(movie_signature[1])-int(r), int(r))])) 

In [49]:
documents_lsh = documents_lsh.map(lambda x : (x[0], [hash(tuple(band)) % 1000 for band in x[1]]))

In [50]:
documents_lsh.take(1)

[('23890098',
  [915,
   959,
   260,
   435,
   932,
   439,
   440,
   451,
   109,
   155,
   653,
   633,
   469,
   582,
   165,
   40,
   665,
   189,
   3,
   218,
   714,
   164])]

In [51]:
candidat_pairs = documents_lsh.collect()

In [52]:
c_pairs = set()

In [53]:
for mov_list1 in candidat_pairs:
    for mov_list2 in candidat_pairs:
        if mov_list2 != mov_list1 and (mov_list1[0],mov_list2[0]) not in c_pairs:
            if any(v1 == v2 for (v1,v2) in zip(mov_list1[1],mov_list2[1])):
                c_pairs = c_pairs | {(mov_list2[0], mov_list1[0])}

In [54]:
def jaccard(x,y):
    x = set(x)
    y = set(y)
    return len(x & y) / len(x | y)

In [66]:
def get_similar_movie(moviename):
    res = []
    similar_movies = [item[0] if item[1] == moviename else item[1] for item in c_pairs if moviename in item]
    movie_shingles = signatures.filter(lambda movie_sig: movie_sig[0] == moviename).take(1)[0][1]
    others_shingles = dict(signatures.filter(lambda movie_shingles : movie_shingles[0] in similar_movies).collect())
    for movie,shingles in others_shingles.items():
        jac_sim = jaccard(movie_shingles, shingles)
        if jac_sim > 0.8 and jac_sim < 0.98:
            res += [movie]
    return res

In [67]:
print(get_similar_movie('23890098'))

0.01532567049808429
0.00816326530612245
0.014388489208633094
[]


# 2.3

In [69]:
s = dict(signatures.collect())

seen_pairs = []
fps = 0
fns = 0
for m1,shingles1 in s.items():
    for m2,shingles2 in s.items():
        if shingles1 != shingles2 and {m2,m1} not in seen_pairs:
            seen_pairs += [{m1,m2}]
            jac_sim = jaccard(shingles1, shingles2)
            if jac_sim > 0.8 and jac_sim < 0.98 and {m1,m2} not in c_pairs:
                fps += 1
            elif jac_sim < 0.8 and {m1,m2} in c_pairs:
                fns += 1
            
print("% of false positives:",fps/len(seen_pairs))
print("% of false positives:",fns/len(seen_pairs))

4950
% of false positives: 0.0
% of false positives: 0.0
